# 文本内容在ES中的存储形式

<center>
    <img src='./img/match_01.png' width='680px'>
</center>

# Match

其中`Match`是`elasticsearch_dsl`中的一个查询类型，它用于在指定字段中搜索包含特定单词或短语的文档。

`Match`查询可以接受一个参数，即要搜索的文本。可以将文本作为字符串传递给`Match`查询的构造函数，也可以使用一个名为`query`的属性来设置文本。`Match`查询还可以设置其他参数，例如字段名称、操作符（默认为OR）、前缀长度、最小相似度等，以更精确地控制查询。

以下是一个使用`Match`查询的示例代码：

In [1]:
from elasticsearch_dsl import Search, connections
from elasticsearch_dsl.query import Match

# 连接到 Elasticsearch
connections.create_connection(hosts=['localhost'])

<Elasticsearch([{'host': 'localhost'}])>

In [2]:
def search_query(search, query, columns):
    """该函数将按照 Query 进行检索并打印出指定字段"""
    s = search.query(query)
    response = s.execute()

    for hit in response.hits:
        for col in columns:
            print(f"{col}: {hit.to_dict().get(col)}")

In [3]:
# 创建一个查询对象
search = Search(index='books')

# 创建一个Match查询
query = Match(title='西游')
search_query(search, query, columns=['title'])

title: 西游记


在上面的示例中，我们使用`Match`查询在`title`字段中搜索包含“西游”的文档。我们将查询添加到查询对象中，并执行查询，然后遍历结果并打印匹配的文档标题。

# MatchAll

> 匹配出全部的数据，可以使用`MatchAll`查询来匹配所有文档。`MatchAll`查询不需要指定任何条件，只需将其作为查询对象即可。

以下是一个简单的例子，展示如何使用`elasticsearch_dsl`中的`MatchAll`查询：

In [4]:
from elasticsearch_dsl.query import MatchAll

In [5]:
query = MatchAll()
search_query(search, query, columns=['title'])

title: 西游记
title: 红楼梦
title: 水浒传
title: 三国演义
title: 创意者的突破
title: 追风筝的人
title: 马拉拉的故事
title: 爱因斯坦传
title: Python
title: Term


使用MatchAll查询可能会返回大量的文档，因此在实际应用中应该谨慎使用。
通常情况下，需要使用其他查询条件来限制搜索结果，以提高搜索效率。

# MatchPhrase

> `MatchPhrase`查询可以用来搜索包含给定短语的文档。与Match查询不同的是，`MatchPhrase`查询不会将查询短语分成单个词项，并只匹配包含短语的文档。

In [6]:
from elasticsearch_dsl.query import MatchPhrase

In [7]:
# 创建一个 MatchPhrase 查询
query = MatchPhrase(title='西游')
search_query(search, query, columns=['title'])

title: 西游记


In [9]:
# 创建一个 MatchPhrase 查询，
# 这段话中不存在 “西游” 这个词所以什么也没匹配出来
query = MatchPhrase(description='西游')
search_query(search, query, columns=['title'])

# MatchPhrasePrefix

> `MatchPhrasePrefix`查询可以用来搜索包含给定词项前缀的文档。与`MatchPhrase`查询类似，`MatchPhrasePrefix`查询会将查询短语作为一个整体来匹配文档，但是它允许查询短语的前缀匹配文档中的词项。

例如，如果我们使用`MatchPhrase`查询搜索短语“Python Elasticsearch”，它只会匹配包含完整短语的文档，而不会匹配包含“Python”或“Elasticsearch”的单独词项的文档。而`MatchPhrasePrefix`查询则可以匹配包含“Py”或“Elastic”前缀的词项的文档。

以下是一个使用MatchPhrasePrefix查询的示例代码：

In [10]:
from elasticsearch_dsl.query import MatchPhrasePrefix

In [11]:
# 中文是以字为分割的，不太有作用
query = MatchPhrasePrefix(description='吴承恩')
search_query(search, query, columns=['description'])

description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。


In [12]:
# 英文则可以根据前缀进行查询
query = MatchPhrasePrefix(description='Py')
search_query(search, query, columns=['description'])

description: Python elastic search


# MatchBoolPrefix

> `MatchBoolPrefix`查询是`Elasticsearch`中的一种组合查询，它将`Match`和`Bool`查询结合在一起，用于搜索包含给定前缀和满足指定条件的文档。在查询的过程中，`MatchBoolPrefix`会将查询短语分成单个词项，并将它们与查询的字段中的词项进行匹配。

以下是一个使用`MatchBoolPrefix`查询的示例代码：

In [14]:
from elasticsearch_dsl.query import MatchBoolPrefix

# 创建一个MatchBoolPrefix查询
query = MatchBoolPrefix(title='Py')
query = query & Match(category='工具')

search_query(search, query, columns=['description'])

description: Python elastic search


在上面的示例中，我们使用`MatchBoolPrefix`查询在`title`字段中搜索以`Py`开头的词项，并且满足`category='工具'`的文档。我们将查询添加到查询对象中，并执行查询，然后遍历结果并打印匹配的文档标题。

# MatchNone

> `MatchNone`查询是`Elasticsearch`中的一种查询方式，它用于查找不包含任何文档的索引。在某些情况下，我们可能需要检查索引是否为空，或者需要在查询结果为空时采取特定的行动，这时`MatchNone`查询就非常有用了。

以下是一个使用`MatchNone`查询的示例代码：

In [15]:
from elasticsearch_dsl.query import MatchNone

In [16]:
# 创建一个MatchNone查询
query = MatchNone()

# 将MatchNone查询添加到查询对象中
s = search.query(query)

# 执行查询
response = s.execute()

# 检查查询结果是否为空
if response.hits.total.value == 0:
    print("Index is empty")

Index is empty


检查查询结果中是否包含任何文档。如果查询结果为空，则说明查询索引是空的。

`MatchNone`查询没有任何参数，因为它的目的只是返回一个空的查询结果。

# MultiMatch

> `MultiMatch`查询支持在多个字段中搜索。

1. `fields`参数指定要搜索的字段列表。
1. `query`用于指定搜索的文本。
1. `minimum_should_match`用于指定最小相似度。

In [17]:
from elasticsearch_dsl.query import MultiMatch

In [18]:
query = MultiMatch(query='西游记', fields=['title', 'description'])
search_query(search, query, columns=['title', 'description'])

title: 西游记
description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。


In [21]:
# 至少匹配到 '西游记' 中两个字的时候输出，可以看到这里有正常的结果输出
query = MultiMatch(query='西游记', fields=['title'], minimum_should_match=2)
search_query(search, query, columns=['title', 'description'])

title: 西游记
description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描绘师徒四人在取经路上所遭遇的种种艰难险阻，反映了人性的善恶、道德观念和宽容精神，成为中国文学的经典之作。


In [22]:
# 匹配到 '西游记' 中4个字的时候输出，此时无结果输出
query = MultiMatch(query='西游记', fields=['title'], minimum_should_match=4)
search_query(search, query, columns=['title', 'description'])

------------